In [ ]:
import torch
import numpy as np
import pandas as pd

train_X = pd.read_csv('X_train.csv')
train_X_tensor = torch.tensor(train_X.values)[:,1:]
train_y = pd.read_csv("y_train.csv")
train_y_tensor = torch.tensor(train_y.values)[:,1:]

test_X = pd.read_csv('X_test.csv')
test_X_tensor = torch.tensor(test_X.values)[:,1:]

------------------------------------------------------- IMPUTATION TIME -------------------------------------------------------

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10, weights="uniform")
imputer.fit(train_X_tensor.numpy())

X_train_post_imp = imputer.transform(train_X_tensor.numpy())

postimp = torch.tensor(X_train_post_imp)

In [ ]:
from sklearn.impute import SimpleImputer

# X = [[1, 2, nan], [3, 4, 3], [nan, 6, 5], [8, 8, 7]]
imputersimple = SimpleImputer(strategy = "mean")
imputersimple.fit_transform(train_X_tensor.numpy())

X_train_post_simple_imp = imputersimple.fit_transform(train_X_tensor.numpy())

postsimpleimp = torch.tensor(X_train_post_simple_imp)

In [ ]:
train_X_tensor.shape

torch.Size([1212, 832])

In [ ]:
train_y_tensor.shape

torch.Size([1212, 1])

----------------------------------------------------------------------OUTLIER DETECTION-------------------------------------------------

In [ ]:
train_y_tensor_np = train_y_tensor.numpy()

In [115]:
from sklearn.ensemble import IsolationForest
clf_isoforest = IsolationForest(random_state=0, bootstrap = True)
post_outlier = clf_isoforest.fit_predict(X_train_post_imp)

In [116]:
for x in range(len(post_outlier)):
  if post_outlier[x]!=1:
    print(x)

39
117
294
311
462
561
648
697
727
769
864
969
986
990


In [117]:
post_outlier+=1

In [118]:
post_outlier = post_outlier/2

In [119]:
X_train_post_imp = X_train_post_imp[post_outlier==1.0]

In [120]:
train_y_tensor_np = train_y_tensor_np[post_outlier==1.0]

In [121]:
X_train_post_imp.shape

(1191, 832)

In [122]:
train_y_tensor_np.shape

(1191, 1)

------------------------------------------------FEATURE SELECTION--------------------------------------------------

In [ ]:
# from sklearn.feature_selection import RFE
# selector = RFE(estimator, n_features_to_select=5, step=1)
# selector = selector.fit(X, y)
# selector.support_array([ True,  True,  True,  True,  True, False, False, False, False,
#        False])
# selector.ranking_array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])

---------------------------------------------------------REGRESSION TIME------------------------------------------------------------


In [123]:
from sklearn.metrics import r2_score
post_imp_np = X_train_post_imp

------------------------------------VANILLA-------------------------------


In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(post_imp_np, train_y_tensor_np)
prediction = reg.predict(post_imp_np)
r2_score(train_y_tensor_np, prediction)

0.09821461403769394

------------------------------------RIDGE-------------------------------


In [124]:
post_imp_np_a = post_imp_np[:1100]
post_imp_np_b = post_imp_np[1100:]
train_y_tensor_np_a = train_y_tensor_np[:1100]
train_y_tensor_np_b = train_y_tensor_np[1100:]

In [125]:
post_imp_np_a.shape

(1100, 832)

In [ ]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1)
clf.fit(post_imp_np_a, train_y_tensor_np_a)
prediction_b = clf.predict(post_imp_np_b)
r2_score(train_y_tensor_np_b , prediction_b)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=5.01519e-48): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


-0.8653809533656422

In [ ]:
clf2 = Ridge(alpha=0.001)
clf2.fit(post_imp_np, train_y_tensor_np)
prediction = clf2.predict(post_imp_np)
r2_score(train_y_tensor_np , prediction)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=8.94417e-51): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


0.8182962709669428

In [ ]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=10)
clf.fit(post_imp_np_a, train_y_tensor_np_a)
prediction_b = clf.predict(post_imp_np_b)
r2_score(train_y_tensor_np_b , prediction_b)

In [ ]:
from sklearn.linear_model import Ridge

pre_imp_np = postsimpleimp.numpy()
train_y_tensor_np = train_y_tensor.numpy()

clf_pre = Ridge(alpha=0.01)
clf_pre.fit(pre_imp_np, train_y_tensor_np)

prediction_pre = clf_pre.predict(pre_imp_np)
r2_score(train_y_tensor_np,prediction_pre)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=8.84637e-50): result may not be accurate.
  overwrite_a=True).T


0.8089185316978174

----------------------------------RIDGE WITH FEATURE SUBSET SELECTION-----------------------------

In [147]:
from sklearn.feature_selection import SelectKBest, f_regression,  mutual_info_regression
fs = SelectKBest(f_regression, k=200) 
post_imp_subset = fs.fit_transform(post_imp_np_a, train_y_tensor_np_a)
post_imp_support= fs.get_support()
post_imp_subset_a = post_imp_np_a[:,post_imp_support]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:293: RuntimeWarning: invalid value encountered in sqrt
  n_samples * X_means ** 2)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom


In [148]:
post_imp_subset_b = post_imp_np_b[:, post_imp_support]

In [142]:
post_imp_subset.shape

(1100, 300)

In [ ]:
clf_subset = Ridge(alpha=10)
clf_subset.fit(post_imp_subset_a, train_y_tensor_np_a)
prediction_b = clf_subset.predict(post_imp_subset_b)
r2_score(train_y_tensor_np_b , prediction_b)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=8.4029e-47): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


0.304098303142631

In [ ]:
post_imp_subset_full = post_imp_np[:, post_imp_support]

In [ ]:
clf_subset_full = Ridge(alpha=10)
clf_subset_full.fit(post_imp_subset_a, train_y_tensor_np_a)
prediction_b = clf_subset_full.predict(post_imp_subset_b)
r2_score(train_y_tensor_np_b , prediction_b)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:156: LinAlgWarning: Ill-conditioned matrix (rcond=8.4029e-47): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


0.304098303142631

In [ ]:
prediction_a = clf_subset.predict(post_imp_subset)
r2_score(train_y_tensor_np_a , prediction_a)

---------------------------------------LASSO--------------------------------------

In [ ]:
from sklearn.linear_model import Lasso

clf_lasso = Lasso(alpha=10)
clf_lasso.fit(post_imp_np_a, train_y_tensor_np_a)

prediction_lasso_b = clf_lasso.predict(post_imp_np_b)
r2_score(train_y_tensor_np_b,prediction_lasso_b)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19544.13097898815, tolerance: 10.526156727272728
  positive)


0.07346675389841417

In [ ]:
clf_lasso = Lasso(alpha=10)
clf_lasso.fit(post_imp_np, train_y_tensor_np)

prediction_lasso = clf_lasso.predict(post_imp_np)
r2_score(train_y_tensor_np,prediction_lasso)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22620.764673319227, tolerance: 11.550564813278012
  positive)


0.638429441685018

In [ ]:
count0 = 0
countnot = 0
for x in clf_lasso.coef_:
  if x==0.0:
    count0+=1
  else:
    countnot+=1
print(count0)
print(countnot)

419
413


In [ ]:
clf_lasso_full = Lasso(alpha=1)
clf_lasso_full.fit(post_imp_np_a, train_y_tensor_np_a)

prediction_lasso = clf_lasso_full.predict(post_imp_np_b)
r2_score(train_y_tensor_np_b,prediction_lasso)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16651.331515758757, tolerance: 10.526156727272728
  positive)


-0.05611627846509526

In [ ]:
clf_lasso_full = Lasso(alpha=40)
clf_lasso_full.fit(post_imp_subset_a, train_y_tensor_np_a)

prediction_lasso_b = clf_lasso_full.predict(post_imp_subset_b)
r2_score(train_y_tensor_np_b,prediction_lasso_b)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.664e+04, tolerance: 1.089e+01
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


0.2194315600256399

------------------------------------------------------ENSEMBLE METHODS-------------------------------------------------------

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
regens = BaggingRegressor(n_estimators=20, random_state=0).fit(post_imp_np_a, train_y_tensor_np_a)
enspredb = regens.predict(post_imp_np_b)
r2_score(train_y_tensor_np_b,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.4962874929710963

In [ ]:
enspreda = regens.predict(post_imp_np_a)
r2_score(train_y_tensor_np_a,enspreda)

0.9161161311884842

In [ ]:
regens3 = BaggingRegressor(n_estimators=100, random_state=0).fit(post_imp_np_a, train_y_tensor_np_a)
enspredb = regens3.predict(post_imp_np_b)
r2_score(train_y_tensor_np_b,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.5065627749265139

In [132]:
post_imp_subset_b.shape

(91, 200)

In [136]:
regens4 = BaggingRegressor(n_estimators=35, random_state=0).fit(post_imp_subset_a, train_y_tensor_np_a)
enspredb = regens4.predict(post_imp_subset_b)
r2_score(train_y_tensor_np_b,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.5291756480872121

In [168]:
from sklearn import tree
regens10 = BaggingRegressor(base_estimator = tree.DecisionTreeRegressor(max_features=0.5), n_estimators=42, random_state=0, oob_score=True).fit(post_imp_np_a[:,post_imp_support], train_y_tensor_np_a)
enspredb = regens10.predict(post_imp_np_b[:,post_imp_support])
r2_score(train_y_tensor_np_b,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.5430624492085476

In [171]:
from sklearn import tree
regens11 = BaggingRegressor(base_estimator = tree.DecisionTreeRegressor(max_features=0.5), n_estimators=42, random_state=0, oob_score=True).fit(post_imp_np[:,post_imp_support], train_y_tensor_np)
enspredb = regens11.predict(post_imp_np[:,post_imp_support])
r2_score(train_y_tensor_np,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9334264960934875

In [ ]:
# from sklearn.feature_selection import RFE
# regens7 = RFE(BaggingRegressor(n_estimators=20, random_state=0), n_features_to_select=220, step=1)
# regens7 = regens7.fit(post_imp_np_a, train_y_tensor_np_a)

# enspredb = regens7.predict(post_imp_np_b)
# r2_score(train_y_tensor_np_b,enspredb)

In [103]:
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
post_a_scaled = scaler.fit_transform(post_imp_subset_a)
post_b_scaled = scaler.fit_transform(post_imp_subset_b)
regens8 = BaggingRegressor(base_estimator=SVR(kernel='linear'),n_estimators=100,  random_state=0).fit(post_a_scaled, train_y_tensor_np_a)
enspredb = regens8.predict(post_b_scaled)
r2_score(train_y_tensor_np_b,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.12611112215108944

In [108]:
scaler = MinMaxScaler()
post_scaled = scaler.fit_transform(post_imp_subset)

regens9 = BaggingRegressor(n_estimators=100, random_state=0).fit(post_scaled, train_y_tensor_np)
enspredb = regens9.predict(post_scaled)
r2_score(train_y_tensor_np,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9366550716932123

In [113]:
post_imp_subset.shape

(1205, 200)

In [93]:
post_imp_subset = post_imp_np[:,post_imp_support]
regens6 = BaggingRegressor(n_estimators=120, max_features=0.7, random_state=0).fit(post_imp_subset, train_y_tensor_np)
enspredb = regens6.predict(post_imp_subset)
r2_score(train_y_tensor_np,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9367787367617159

In [ ]:
regens5 = BaggingRegressor(n_estimators=40, random_state=0).fit(post_imp_np, train_y_tensor_np)
enspredb = regens5.predict(post_imp_np)
r2_score(train_y_tensor_np,enspredb)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9277037085632341

-------------------------------------------------------------MLP------------------------------------------------------------

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

regr = MLPRegressor(random_state=1, max_iter=100000).fit(post_imp_np, train_y_tensor_np[:,0])

prediction_mlp_a =regr.predict(post_imp_np)
r2_score(train_y_tensor_np[:,0],prediction_mlp_a)

-2.4766788590125114e+31

-----------------------------------------------------------TABNET--------------------------------------------------------

In [ ]:
X_tabnet_a = post_imp_np[:1150]
X_tabnet_b = post_imp_np[1150:1200]
X_tabnet_c = post_imp_np[1190:]
y_tabnet_a = train_y_tensor_np[:1150]
y_tabnet_b = train_y_tensor_np[1150:1190]
y_tabnet_c = train_y_tensor_np[1190:]

In [ ]:
pip install pytorch-tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

clf = TabNetRegressor()
clf.fit(
  X_tabnet_a, y_tabnet_a,
  eval_set=[(X_tabnet_b, y_tabnet_b)]
)
tabnet_preds = clf.predict(X_tabnet_c)
r2_score(y_tabnet_c, tabnet_preds)

Device used : cpu
epoch 0  | loss: 4789.60035| val_0_mse: 841617.90682|  0:00:00s
epoch 1  | loss: 4668.49722| val_0_mse: 16406583002.70559|  0:00:01s
epoch 2  | loss: 4554.29857| val_0_mse: 1800.27952|  0:00:01s
epoch 3  | loss: 4429.74551| val_0_mse: 5128.21158|  0:00:02s
epoch 4  | loss: 4313.58877| val_0_mse: 395455665981.3328|  0:00:03s
epoch 5  | loss: 4186.68046| val_0_mse: 249537179110.12085|  0:00:03s
epoch 6  | loss: 4027.90663| val_0_mse: 165238065784.23053|  0:00:04s
epoch 7  | loss: 3872.06001| val_0_mse: 40490.78788|  0:00:04s
epoch 8  | loss: 3720.3479| val_0_mse: 87264.15277|  0:00:05s
epoch 9  | loss: 3530.28481| val_0_mse: 152837.75645|  0:00:06s
epoch 10 | loss: 3368.27881| val_0_mse: 93544.4635|  0:00:06s
epoch 11 | loss: 3153.02121| val_0_mse: 188019.28927|  0:00:07s
epoch 12 | loss: 2958.15213| val_0_mse: 60147.618|  0:00:07s

Early stopping occurred at epoch 12 with best_epoch = 2 and best_val_0_mse = 1800.27952
Best weights from best epoch are automatically used

-8.747738229803117

PREDICTING THE TEST VALUES


In [59]:
# X = [[1, 2, nan], [3, 4, 3], [nan, 6, 5], [8, 8, 7]]
imputertest = KNNImputer(n_neighbors=10, weights="uniform")

imputertest.fit(test_X_tensor.numpy())

X_test_post_imp = imputertest.transform(test_X_tensor.numpy())

In [ ]:
predictiontest = clf.predict(X_test_post_imp)

In [94]:
predictiontest = regens6.predict(X_test_post_imp[:,post_imp_support])

In [172]:
predictiontest = regens11.predict(X_test_post_imp[:,post_imp_support])

In [109]:
scaler = MinMaxScaler()
post_scaled = scaler.fit_transform(X_test_post_imp[:,post_imp_support])
predictiontest = regens9.predict(post_scaled)

In [173]:
test_y = pd.read_csv('sample.csv')
test_y.loc[:,"y"] = predictiontest
test_y.to_csv("predictions.csv", index=False)

In [ ]:
test_y.columns

Index(['id', 'y'], dtype='object')